In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import random
import os
import math

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 42
seed_everything(SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {DEVICE}")

# ======================================================
# [V12-Lite] Type/Result Embedding (희소성 문제 해결)
# ======================================================
BATCH_SIZE = 64
LR_BASE = 1e-3
WARMUP_EPOCHS = 3
EPOCHS_BASE = 50
DROPOUT = 0.2
MAX_SEQ_LEN = 30
GRAD_CLIP = 1.0

HIDDEN_DIM = 256
LSTM_LAYERS = 3
BIDIRECTIONAL = True

# MDN 파라미터
NUM_GAUSSIANS = 2
MIN_SIGMA = 0.01
MAX_SIGMA = 0.3
HYBRID_LOSS_WEIGHT = 0.3

print(f"[V12-Lite] Type/Result Embedding (개선 버전)")
print(f"  주요 변경사항:")
print(f"  1. Type/Result만 Embedding (희소성 문제 해결)")
print(f"  2. Player → is_same 피처 활용")
print(f"  3. Team → is_home 피처 활용")
print(f"  4. 작은 초기화 (std=0.01)")
print(f"  5. Embedding Dropout 추가")
print(f"  6. 예상: V10 대비 1-2m 개선")

In [ ]:
# 데이터 로드 (V11과 동일)
BASE_DIR = "./open_track1"
if not os.path.exists(BASE_DIR): BASE_DIR = "."

TRAIN_PATH = os.path.join(BASE_DIR, "train.csv")
TEST_META_PATH = os.path.join(BASE_DIR, "test.csv")
MATCH_PATH = os.path.join(BASE_DIR, "match_info.csv")

train_df = pd.read_csv(TRAIN_PATH)
print(f"✅ Train Loaded: {train_df.shape}")

if os.path.exists(TEST_META_PATH):
    test_meta = pd.read_csv(TEST_META_PATH)
    print(f"ℹ️ Reading {len(test_meta)} test files...")
    test_dfs = []
    for _, row in tqdm(test_meta.iterrows(), total=len(test_meta), desc="Loading Test CSVs"):
        rel_path = row['path']
        paths_to_try = [
            rel_path,
            os.path.join(BASE_DIR, rel_path.lstrip("./")),
            os.path.join(BASE_DIR, "test", str(row['game_id']), os.path.basename(rel_path))
        ]
        for p in paths_to_try:
            if os.path.exists(p):
                test_dfs.append(pd.read_csv(p))
                break
    if test_dfs:
        test_df = pd.concat(test_dfs, ignore_index=True)
        print(f"✅ Test Data Merged: {test_df.shape}")
else:
    raise FileNotFoundError("test.csv not found")

if os.path.exists(MATCH_PATH):
    match_info = pd.read_csv(MATCH_PATH)
    match_subset = match_info[['game_id', 'home_team_id', 'venue']]
    train_df = pd.merge(train_df, match_subset, on='game_id', how='left')
    test_df = pd.merge(test_df, match_subset, on='game_id', how='left')

def preprocess(df):
    if 'home_team_id' in df.columns:
        df['is_home'] = (df['team_id'] == df['home_team_id']).astype(float)
    else:
        df['is_home'] = 0.5
    if 'end_x' not in df.columns:
        df['end_x'] = 0.0
        df['end_y'] = 0.0
    else:
        df['end_x'] = df['end_x'].fillna(0.0)
        df['end_y'] = df['end_y'].fillna(0.0)
    return df

train_df = preprocess(train_df)
test_df = preprocess(test_df)

ID_COL = 'game_episode' if 'game_episode' in train_df.columns else 'episode_id'
print(f"Data Ready. ID Column: {ID_COL}")

# ======================================================
# [Entity Embedding] 매핑 딕셔너리 생성
# ======================================================
print("\n" + "="*70)
print("🔧 [Entity Embedding] 매핑 딕셔너리 생성")
print("="*70)

# Player ID 매핑 (0은 padding/unknown)
all_player_ids = pd.concat([train_df['player_id'], test_df['player_id']]).dropna().unique()
player_to_idx = {pid: i+1 for i, pid in enumerate(sorted(all_player_ids))}
player_to_idx[0] = 0  # Unknown/padding
NUM_PLAYERS = len(player_to_idx)

# Team ID 매핑 (0은 unknown)
all_team_ids = pd.concat([train_df['team_id'], test_df['team_id']]).dropna().unique()
team_to_idx = {tid: i+1 for i, tid in enumerate(sorted(all_team_ids))}
team_to_idx[0] = 0
NUM_TEAMS = len(team_to_idx)

# Type Name 매핑 (마지막 인덱스는 unknown)
type_to_idx = {t: i for i, t in enumerate(TOP_TYPES)}
type_to_idx['unknown'] = len(TOP_TYPES)
NUM_TYPES = len(TOP_TYPES) + 1

# Result Name 매핑 (마지막 인덱스는 unknown)
result_to_idx = {r: i for i, r in enumerate(ALL_RESULTS)}
result_to_idx['unknown'] = len(ALL_RESULTS)
NUM_RESULTS = len(ALL_RESULTS) + 1

print(f"✅ Players: {NUM_PLAYERS} (unique: {len(all_player_ids)})")
print(f"✅ Teams: {NUM_TEAMS} (unique: {len(all_team_ids)})")
print(f"✅ Types: {NUM_TYPES} (categories: {len(TOP_TYPES)})")
print(f"✅ Results: {NUM_RESULTS} (categories: {len(ALL_RESULTS)})")
print("="*70)

In [ ]:
# 피처 엔지니어링 (Entity Embedding 버전 - Type/Result만)
TOP_TYPES = ['Pass', 'Carry', 'Recovery', 'Interception', 'Duel', 'Tackle', 
             'Throw-In', 'Clearance', 'Intervention', 'Block', 'Pass_Freekick', 
             'Cross', 'Goal Kick', 'Error', 'Shot']
ALL_RESULTS = ['Successful', 'Unsuccessful', 'On Target', 'Yellow_Card', 
               'Blocked', 'Keeper Rush-Out', 'Low Quality Shot', 'Off Target']

def make_features_embedding(group):
    """
    [Entity Embedding] Type/Result만 Embedding, Player/Team은 기존 피처 활용
    Returns:
        continuous_features: (seq_len, num_continuous)
        categorical_indices: (seq_len, 2) - [type_idx, result_idx]
    """
    n = len(group)
    sx = group['start_x'].values / 105.0
    sy = group['start_y'].values / 68.0
    ex = group['end_x'].values / 105.0
    ey = group['end_y'].values / 68.0
    is_home = group['is_home'].values
    
    if 'time_seconds' in group.columns:
        times = group['time_seconds'].values
        dt = np.zeros(n, dtype=np.float32)
        dt[1:] = times[1:] - times[:-1]
        dt = np.maximum(dt, 0.1)
    else:
        dt = np.ones(n, dtype=np.float32)

    dx = ex - sx
    dy = ey - sy
    dist_meter = np.sqrt((dx*105)**2 + (dy*68)**2)
    cumsum_dx = np.cumsum(dx) / 105.0
    cumsum_dy = np.cumsum(dy) / 68.0
    lag_dist_m = np.roll(dist_meter, 1); lag_dist_m[0] = 0
    lag_cumsum_dx = np.roll(cumsum_dx, 1); lag_cumsum_dx[0] = 0
    lag_cumsum_dy = np.roll(cumsum_dy, 1); lag_cumsum_dy[0] = 0
    lag_dt = np.roll(dt, 1); lag_dt[0] = 1.0
    lag_speed = lag_dist_m / np.maximum(lag_dt, 0.1)
    
    if 'player_id' in group.columns:
        p_ids = group['player_id'].values
        is_same = np.zeros(n, dtype=np.float32)
        is_same[1:] = (p_ids[1:] == p_ids[:-1]).astype(np.float32)
    else:
        is_same = np.zeros(n, dtype=np.float32)

    progress = np.arange(n) / max(n-1, 1)
    is_second_half = (group['period_id'].values > 1).astype(np.float32) if 'period_id' in group.columns else np.zeros(n)
    
    GOAL_X, GOAL_Y = 105.0, 34.0
    sx_real, sy_real = sx * 105.0, sy * 68.0
    dist_to_goal = np.sqrt((sx_real - GOAL_X)**2 + (sy_real - GOAL_Y)**2) / 105.0
    angle_to_goal = np.arctan2(GOAL_Y - sy_real, GOAL_X - sx_real)
    angle_sin, angle_cos = np.sin(angle_to_goal), np.cos(angle_to_goal)
    dist_to_sideline = np.minimum(sy_real, 68.0 - sy_real) / 68.0
    dist_to_endline = np.minimum(sx_real, 105.0 - sx_real) / 105.0
    
    def get_zone(x_norm):
        if x_norm < 35.0/105.0: return 0
        elif x_norm < 70.0/105.0: return 1
        else: return 2
    
    zones = np.array([get_zone(x) for x in sx])
    zone_onehot = np.zeros((n, 3), dtype=np.float32)
    for i, z in enumerate(zones): zone_onehot[i, z] = 1.0
    
    # ✅ Categorical Indices (Type/Result만)
    type_indices = np.array([type_to_idx.get(t, type_to_idx['unknown']) for t in group['type_name'].values], dtype=np.int64)
    result_indices = np.array([result_to_idx.get(r, result_to_idx['unknown']) for r in group['result_name'].values], dtype=np.int64)

    continuous_features = []
    categorical_features = []
    
    for i in range(n):
        # Continuous features
        scalars = [sx[i], sy[i], lag_cumsum_dx[i], lag_cumsum_dy[i], lag_dist_m[i]/100.0,
                   lag_speed[i]/10.0, dt[i]/10.0, progress[i], is_home[i], is_same[i],
                   is_second_half[i], dist_to_goal[i], angle_sin[i], angle_cos[i],
                   dist_to_sideline[i], dist_to_endline[i]]
        continuous_vec = np.concatenate([scalars, zone_onehot[i]])  # 16 + 3 = 19
        continuous_features.append(continuous_vec)
        
        # Categorical indices (Type/Result만)
        categorical_features.append([type_indices[i], result_indices[i]])
        
        # End state features
        if i < n - 1:
            ex_real, ey_real = ex[i] * 105.0, ey[i] * 68.0
            end_dist_to_goal = np.sqrt((ex_real - GOAL_X)**2 + (ey_real - GOAL_Y)**2) / 105.0
            end_angle = np.arctan2(GOAL_Y - ey_real, GOAL_X - ex_real)
            scalars_end = scalars.copy()
            scalars_end[0:2] = [ex[i], ey[i]]
            scalars_end[2:4] = [cumsum_dx[i], cumsum_dy[i]]
            scalars_end[11:16] = [end_dist_to_goal, np.sin(end_angle), np.cos(end_angle),
                                   min(ey_real, 68.0 - ey_real) / 68.0,
                                   min(ex_real, 105.0 - ex_real) / 105.0]
            end_zone_onehot = np.zeros(3, dtype=np.float32)
            end_zone_onehot[get_zone(ex[i])] = 1.0
            continuous_vec_end = np.concatenate([scalars_end, end_zone_onehot])
            continuous_features.append(continuous_vec_end)
            
            # Same categorical for end state
            categorical_features.append([type_indices[i], result_indices[i]])
            
    return np.array(continuous_features, dtype=np.float32), np.array(categorical_features, dtype=np.int64)

# Test
dummy_group = train_df.iloc[:5].copy()
cont_feat, cat_feat = make_features_embedding(dummy_group)
CONTINUOUS_DIM = cont_feat.shape[1]
print(f"✅ Continuous Dim: {CONTINUOUS_DIM}")
print(f"✅ Categorical Dim: {cat_feat.shape[1]} (type, result only)")
print(f"✅ Sample continuous: {cont_feat[0, :5]}")
print(f"✅ Sample categorical: {cat_feat[0]}")

In [ ]:
# 데이터셋 (Entity Embedding 버전)
class SoccerDatasetEmbedding(Dataset):
    def __init__(self, df, mode='train'):
        self.mode = mode
        self.continuous_seqs, self.categorical_seqs = [], []
        self.targets, self.team_ids, self.episode_ids = [], [], []
        
        for name, group in tqdm(df.groupby(ID_COL, sort=False), desc=f"Dataset ({mode})"):
            if mode == 'train' and len(group) < 2: continue
            
            cont_seq, cat_seq = make_features_embedding(group)
            team_id = group.iloc[0]['team_id']
            
            if mode == 'train':
                last = group.iloc[-1]
                self.targets.append([last['end_x']/105.0, last['end_y']/68.0])
                self.continuous_seqs.append(cont_seq)
                self.categorical_seqs.append(cat_seq)
                self.team_ids.append(team_id)
            else:
                self.continuous_seqs.append(cont_seq)
                self.categorical_seqs.append(cat_seq)
                self.team_ids.append(team_id)
                self.episode_ids.append(str(name))

    def __len__(self): 
        return len(self.continuous_seqs)
    
    def __getitem__(self, idx):
        cont = torch.FloatTensor(self.continuous_seqs[idx])
        cat = torch.LongTensor(self.categorical_seqs[idx])
        
        if len(cont) > MAX_SEQ_LEN:
            cont = cont[-MAX_SEQ_LEN:]
            cat = cat[-MAX_SEQ_LEN:]
        
        if self.mode == 'train':
            return cont, cat, torch.FloatTensor(self.targets[idx]), self.team_ids[idx]
        return cont, cat, self.team_ids[idx], self.episode_ids[idx]

def collate_fn_embedding(batch):
    """
    Collate function for embedding dataset
    Returns:
        - continuous: padded (batch, max_len, continuous_dim)
        - categorical: padded (batch, max_len, 4)
        - targets (train only)
        - mask
        - lengths
        - team_ids
        - episode_ids (test only)
    """
    continuous_seqs = [b[0] for b in batch]
    categorical_seqs = [b[1] for b in batch]
    lens = torch.LongTensor([len(s) for s in continuous_seqs])
    
    # Padding
    continuous_padded = pad_sequence(continuous_seqs, batch_first=True, padding_value=0)
    categorical_padded = pad_sequence(categorical_seqs, batch_first=True, padding_value=0)
    
    mask = torch.arange(continuous_padded.size(1))[None, :] >= lens[:, None]
    
    if isinstance(batch[0][2], str):  # Test mode
        return continuous_padded, categorical_padded, mask, lens, torch.LongTensor([b[2] for b in batch]), [b[3] for b in batch]
    else:  # Train mode
        return continuous_padded, categorical_padded, torch.stack([b[2] for b in batch]), mask, lens, torch.LongTensor([b[3] for b in batch])

full_dataset = SoccerDatasetEmbedding(train_df, mode='train')
test_dataset = SoccerDatasetEmbedding(test_df, mode='test')
print(f"✅ Dataset: {len(full_dataset)} train, {len(test_dataset)} test")

In [ ]:
# ======================================================
# [V12-Entity Embedding-Lite] Type/Result만 Embedding
# ======================================================

# Embedding Dimensions (Type/Result만)
TYPE_EMBED_DIM = 12
RESULT_EMBED_DIM = 6

class SpatialAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.goal_attn = nn.Sequential(nn.Linear(3, 16), nn.ReLU(), nn.Linear(16, 1))
        self.zone_attn = nn.Sequential(nn.Linear(3, 8), nn.ReLU(), nn.Linear(8, 1))
        self.pos_attn = nn.Sequential(nn.Linear(2, 8), nn.ReLU(), nn.Linear(8, 1))
        self.fusion = nn.Linear(3, 1)
    def forward(self, x):
        return self.fusion(torch.cat([self.pos_attn(x[..., 0:2]), 
                                       self.goal_attn(x[..., 11:14]), 
                                       self.zone_attn(x[..., 16:19])], dim=-1))

class TemporalAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.pos_encoding = nn.Parameter(torch.randn(1, 100, hidden_dim) * 0.02)
        self.temporal_attn = nn.Sequential(nn.Linear(hidden_dim, hidden_dim // 2), 
                                           nn.Tanh(), nn.Dropout(0.1), 
                                           nn.Linear(hidden_dim // 2, 1))
    def forward(self, lstm_out):
        return self.temporal_attn(lstm_out + self.pos_encoding[:, :lstm_out.size(1), :])

class SpatialTemporalFusion(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.spatial_weight = nn.Parameter(torch.tensor(0.5))
        self.temporal_weight = nn.Parameter(torch.tensor(0.5))
        self.combine = nn.Sequential(nn.Linear(2, 8), nn.ReLU(), nn.Linear(8, 1), nn.Sigmoid())
    def forward(self, s, t):
        return (torch.sigmoid(self.spatial_weight) * s + 
                torch.sigmoid(self.temporal_weight) * t) * self.combine(torch.cat([s, t], -1))

class MDNPredictorWithEmbeddingLite(nn.Module):
    """
    [V12-Lite] Type/Result만 Embedding (Player/Team 제거)
    - 희소성 문제 해결
    - 안정적인 학습
    """
    def __init__(self, continuous_dim, hidden_dim, num_layers, dropout, num_gaussians=2, bidirectional=True):
        super().__init__()
        self.num_gaussians = num_gaussians
        
        # ✅ Type/Result Embedding만 사용
        self.type_embedding = nn.Embedding(NUM_TYPES, TYPE_EMBED_DIM)
        self.result_embedding = nn.Embedding(NUM_RESULTS, RESULT_EMBED_DIM)
        
        # ✅ 작은 값으로 초기화 (Xavier 대신)
        nn.init.normal_(self.type_embedding.weight, mean=0, std=0.01)
        nn.init.normal_(self.result_embedding.weight, mean=0, std=0.01)
        
        # ✅ Embedding Dropout 추가
        self.embedding_dropout = nn.Dropout(dropout * 0.5)
        
        # Total input dimension
        total_input_dim = continuous_dim + TYPE_EMBED_DIM + RESULT_EMBED_DIM
        
        self.lstm = nn.LSTM(total_input_dim, hidden_dim, num_layers, batch_first=True,
                           dropout=dropout if num_layers > 1 else 0, bidirectional=bidirectional)
        lstm_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        
        self.spatial_attn = SpatialAttention(lstm_output_dim)
        self.temporal_attn = TemporalAttention(lstm_output_dim)
        self.fusion = SpatialTemporalFusion(lstm_output_dim)
        
        # MDN Heads
        self.pi_head = nn.Sequential(
            nn.Linear(lstm_output_dim, lstm_output_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_output_dim // 2, num_gaussians)
        )
        self.mu_head = nn.Sequential(
            nn.Linear(lstm_output_dim, lstm_output_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_output_dim // 2, num_gaussians * 2)
        )
        self.sigma_head = nn.Sequential(
            nn.Linear(lstm_output_dim, lstm_output_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_output_dim // 2, num_gaussians * 2)
        )
        
        # Mu 초기화
        nn.init.constant_(self.mu_head[-1].bias, 0.5)
        nn.init.xavier_uniform_(self.mu_head[-1].weight, gain=0.1)

    def forward(self, continuous, categorical, mask=None, lengths=None):
        """
        Args:
            continuous: (batch, seq_len, continuous_dim)
            categorical: (batch, seq_len, 2) - [type_idx, result_idx]
        """
        batch_size, seq_len = continuous.size(0), continuous.size(1)
        
        # ✅ Embedding lookup (Type/Result만)
        type_idx = categorical[:, :, 0]
        result_idx = categorical[:, :, 1]
        
        type_emb = self.type_embedding(type_idx)
        result_emb = self.result_embedding(result_idx)
        
        # ✅ Embedding Dropout 적용
        type_emb = self.embedding_dropout(type_emb)
        result_emb = self.embedding_dropout(result_emb)
        
        # ✅ Concatenate continuous + embeddings
        x = torch.cat([continuous, type_emb, result_emb], dim=-1)
        
        # LSTM + Attention
        if lengths is not None:
            packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
            lstm_out, _ = self.lstm(packed)
            lstm_out, _ = pad_packed_sequence(lstm_out, batch_first=True, total_length=seq_len)
        else:
            lstm_out, _ = self.lstm(x)
        
        fused_attn = self.fusion(self.spatial_attn(continuous), self.temporal_attn(lstm_out))
        if mask is not None:
            fused_attn = fused_attn.masked_fill(mask.unsqueeze(-1), float('-inf'))
        final_attn = torch.softmax(fused_attn, dim=1)
        context = torch.sum(lstm_out * final_attn, dim=1)
        
        # MDN 출력
        pi = torch.softmax(self.pi_head(context), dim=1)
        mu = torch.sigmoid(self.mu_head(context)).view(batch_size, self.num_gaussians, 2)
        
        sigma_raw = self.sigma_head(context).view(batch_size, self.num_gaussians, 2)
        sigma = torch.sigmoid(sigma_raw) * (MAX_SIGMA - MIN_SIGMA) + MIN_SIGMA
        
        return pi, mu, sigma


def hybrid_mdn_loss(pi, mu, sigma, target, mse_weight=HYBRID_LOSS_WEIGHT):
    """Hybrid Loss: NLL + MSE"""
    batch_size = target.size(0)
    target_expanded = target.unsqueeze(1).expand_as(mu)
    
    # NLL Loss
    diff = target_expanded - mu
    log_prob_components = (
        -0.5 * math.log(2 * math.pi) - torch.log(sigma) - 0.5 * (diff / sigma) ** 2
    )
    log_prob = log_prob_components.sum(dim=2)
    weighted_log_prob = log_prob + torch.log(pi + 1e-8)
    nll_loss = -torch.logsumexp(weighted_log_prob, dim=1).mean()
    
    # MSE Loss
    pred_mean = (pi.unsqueeze(-1) * mu).sum(dim=1)
    mse_loss = nn.functional.mse_loss(pred_mean, target)
    
    total_loss = nll_loss + mse_weight * mse_loss
    
    return total_loss, nll_loss, mse_loss


def mdn_predict_improved(pi, mu, sigma, strategy='mean'):
    """개선된 예측"""
    if strategy == 'mode':
        max_idx = torch.argmax(pi, dim=1)
        pred = mu[torch.arange(len(mu)), max_idx]
    elif strategy == 'mean':
        pred = (pi.unsqueeze(-1) * mu).sum(dim=1)
    else:
        raise ValueError(f"Unknown strategy: {strategy}")
    
    return torch.clamp(pred, 0.0, 1.0)


model = MDNPredictorWithEmbeddingLite(CONTINUOUS_DIM, HIDDEN_DIM, LSTM_LAYERS, DROPOUT, NUM_GAUSSIANS, BIDIRECTIONAL).to(DEVICE)

print("="*70)
print("✅ [V12-Lite] Type/Result Embedding 모델 준비 완료")
print("="*70)
print(f"Embedding 구성:")
print(f"  - Type: {NUM_TYPES} → {TYPE_EMBED_DIM}dim")
print(f"  - Result: {NUM_RESULTS} → {RESULT_EMBED_DIM}dim")
print(f"  - Player: is_same 피처로 대체 (Embedding 제거)")
print(f"  - Team: is_home 피처로 대체 (Embedding 제거)")
print(f"  - Total Embedding: {TYPE_EMBED_DIM + RESULT_EMBED_DIM}dim")
print(f"  - Continuous: {CONTINUOUS_DIM}dim")
print(f"  - LSTM Input: {CONTINUOUS_DIM + TYPE_EMBED_DIM + RESULT_EMBED_DIM}dim")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"\n개선사항:")
print(f"  1. 희소성 문제 해결 (Player/Team 제거)")
print(f"  2. 작은 초기화 (std=0.01)")
print(f"  3. Embedding Dropout 추가")
print(f"  4. 안정적인 학습 보장")
print("="*70)

In [ ]:
# ======================================================
# [V12-Lite] 학습
# ======================================================
from torch.optim.lr_scheduler import CosineAnnealingLR

SEEDS = [42, 2024, 777]
PRED_STRATEGY = 'mean'

print(f"\n{'='*70}")
print(f"🚀 [V12-Lite] Type/Result Embedding 학습 시작")
print(f"{'='*70}\n")

all_histories = []

for i, seed in enumerate(SEEDS):
    print(f"\n{'='*70}")
    print(f"📦 [모델 {i+1}/3] Seed {seed}")
    print(f"{'='*70}")
    seed_everything(seed)
    
    train_idx, val_idx = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=seed)
    train_loader = DataLoader(torch.utils.data.Subset(full_dataset, train_idx), 
                              batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn_embedding)
    val_loader = DataLoader(torch.utils.data.Subset(full_dataset, val_idx), 
                           batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn_embedding)
    
    model = MDNPredictorWithEmbeddingLite(CONTINUOUS_DIM, HIDDEN_DIM, LSTM_LAYERS, DROPOUT, 
                                          NUM_GAUSSIANS, BIDIRECTIONAL).to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=LR_BASE, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS_BASE - WARMUP_EPOCHS, eta_min=1e-6)
    
    best_dist = float('inf')
    history = {'train_loss': [], 'train_nll': [], 'train_mse': [], 'val_dist': [], 'lr': []}
    
    for epoch in range(EPOCHS_BASE):
        # Warmup
        if epoch < WARMUP_EPOCHS:
            for param_group in optimizer.param_groups:
                param_group['lr'] = LR_BASE * (epoch + 1) / WARMUP_EPOCHS
        
        # Training
        model.train()
        train_loss, train_nll, train_mse = 0.0, 0.0, 0.0
        for continuous, categorical, targets, mask, lens, _ in train_loader:
            continuous = continuous.to(DEVICE)
            categorical = categorical.to(DEVICE)
            targets = targets.to(DEVICE)
            mask = mask.to(DEVICE)
            lens = lens.to(DEVICE)
            
            optimizer.zero_grad()
            pi, mu, sigma = model(continuous, categorical, mask, lens)
            loss, nll, mse = hybrid_mdn_loss(pi, mu, sigma, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            optimizer.step()
            
            train_loss += loss.item()
            train_nll += nll.item()
            train_mse += mse.item()
        
        # Validation
        model.eval()
        dists = []
        with torch.no_grad():
            for continuous, categorical, targets, mask, lens, _ in val_loader:
                continuous = continuous.to(DEVICE)
                categorical = categorical.to(DEVICE)
                targets = targets.to(DEVICE)
                mask = mask.to(DEVICE)
                lens = lens.to(DEVICE)
                
                pi, mu, sigma = model(continuous, categorical, mask, lens)
                pred = mdn_predict_improved(pi, mu, sigma, strategy=PRED_STRATEGY)
                
                p_real = pred.cpu().numpy() * np.array([105.0, 68.0])
                t_real = targets.cpu().numpy() * np.array([105.0, 68.0])
                dists.extend(np.sqrt(np.sum((p_real - t_real)**2, axis=1)))
        
        avg_dist = np.mean(dists)
        avg_loss = train_loss / len(train_loader)
        avg_nll = train_nll / len(train_loader)
        avg_mse = train_mse / len(train_loader)
        
        if epoch >= WARMUP_EPOCHS:
            scheduler.step()
        
        current_lr = optimizer.param_groups[0]['lr']
        history['train_loss'].append(avg_loss)
        history['train_nll'].append(avg_nll)
        history['train_mse'].append(avg_mse)
        history['val_dist'].append(avg_dist)
        history['lr'].append(current_lr)
        
        if (epoch + 1) % 5 == 0 or epoch < 5:
            print(f"  [Epoch {epoch+1:2d}/{EPOCHS_BASE}] "
                  f"Loss: {avg_loss:.4f} (NLL: {avg_nll:.4f}, MSE: {avg_mse:.4f}) | "
                  f"Val: {avg_dist:.4f}m | LR: {current_lr:.6f}")
        
        if avg_dist < best_dist:
            best_dist = avg_dist
            torch.save(model.state_dict(), f'v12_lite_mdn_{i}.pth')
    
    history['best_dist'] = best_dist
    history['seed'] = seed
    all_histories.append(history)
    print(f"  ✅ Best: {best_dist:.4f}m")

print(f"\n{'='*70}")
print(f"✅ 학습 완료!")
print(f"{'='*70}")
print(f"평균 Best: {np.mean([h['best_dist'] for h in all_histories]):.4f}m")
print(f"최고 성능: {min([h['best_dist'] for h in all_histories]):.4f}m")
print(f"{'='*70}")

In [ ]:
# 추론
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn_embedding)

models = []
for i in range(3):
    model = MDNPredictorWithEmbeddingLite(CONTINUOUS_DIM, HIDDEN_DIM, LSTM_LAYERS, DROPOUT, 
                                          NUM_GAUSSIANS, BIDIRECTIONAL).to(DEVICE)
    model.load_state_dict(torch.load(f'v12_lite_mdn_{i}.pth'))
    model.eval()
    models.append(model)

results = []
with torch.no_grad():
    for continuous, categorical, mask, lens, team_ids, episode_ids in tqdm(test_loader, desc="Inference"):
        continuous = continuous.to(DEVICE)
        categorical = categorical.to(DEVICE)
        mask = mask.to(DEVICE)
        lens = lens.to(DEVICE)
        
        preds = []
        for model in models:
            pi, mu, sigma = model(continuous, categorical, mask, lens)
            pred = mdn_predict_improved(pi, mu, sigma, strategy=PRED_STRATEGY)
            preds.append(pred.cpu().numpy())
        
        avg_pred = np.mean(preds, axis=0)
        for i, eid in enumerate(episode_ids):
            results.append({'game_episode': eid, 'pred_x': avg_pred[i, 0] * 105.0, 
                          'pred_y': avg_pred[i, 1] * 68.0})

pred_df = pd.DataFrame(results)
SUBMISSION_PATH = "open_track1/sample_submission.csv"
if os.path.exists(SUBMISSION_PATH):
    sub = pd.read_csv(SUBMISSION_PATH)
else:
    sub = pd.read_csv(TEST_META_PATH).rename(columns={'episode_id': 'game_episode'})[['game_episode']]

final_sub = pd.merge(sub[['game_episode']], pred_df, on='game_episode', how='left')
final_sub = final_sub.rename(columns={'pred_x': 'end_x', 'pred_y': 'end_y'})
if final_sub.isnull().sum().sum() > 0:
    final_sub = final_sub.fillna(50.0)

filename = "v12_lite_embedding_submit.csv"
final_sub.to_csv(filename, index=False)

print(f"\n✅ 제출 파일: {filename}")
print(final_sub.head())

In [ ]:
# 시각화
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('[V12-Lite] Type/Result Embedding 학습 곡선', fontsize=16, fontweight='bold')

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

# 1. Hybrid Loss
ax1 = axes[0, 0]
for i, h in enumerate(all_histories):
    epochs = range(1, len(h['train_loss']) + 1)
    ax1.plot(epochs, h['train_loss'], label=f"Model {i+1}", color=colors[i], linewidth=2)
ax1.set_title('Training Loss (Hybrid: NLL + MSE)')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Validation Distance
ax2 = axes[0, 1]
for i, h in enumerate(all_histories):
    epochs = range(1, len(h['val_dist']) + 1)
    ax2.plot(epochs, h['val_dist'], label=f"Model {i+1}", color=colors[i], linewidth=2)
    best_idx = np.argmin(h['val_dist'])
    ax2.scatter(best_idx + 1, h['val_dist'][best_idx], color=colors[i], s=100, marker='*')
ax2.set_title(f'Validation Distance ({PRED_STRATEGY})')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Distance (m)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. NLL vs MSE
ax3 = axes[1, 0]
h = all_histories[0]
epochs = range(1, len(h['train_nll']) + 1)
ax3.plot(epochs, h['train_nll'], label='NLL', color='red', linewidth=2)
ax3_twin = ax3.twinx()
ax3_twin.plot(epochs, h['train_mse'], label='MSE', color='blue', linewidth=2)
ax3.set_xlabel('Epoch')
ax3.set_ylabel('NLL', color='red')
ax3_twin.set_ylabel('MSE', color='blue')
ax3.set_title('Loss Components (Model 1)')
ax3.grid(True, alpha=0.3)

# 4. Best Distances
ax4 = axes[1, 1]
best_dists = [h['best_dist'] for h in all_histories]
bars = ax4.bar(range(len(best_dists)), best_dists, color=colors)
ax4.set_xticks(range(len(best_dists)))
ax4.set_xticklabels([f'Model {i+1}' for i in range(len(best_dists))])
ax4.set_ylabel('Best Distance (m)')
ax4.set_title('Best Performance')
for bar, dist in zip(bars, best_dists):
    ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height(), 
             f'{dist:.4f}m', ha='center', va='bottom', fontweight='bold')
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('v12_lite_embedding_history.png', dpi=150)
plt.show()

print(f"\n최종 결과:")
for i, h in enumerate(all_histories):
    print(f"  Model {i+1}: {h['best_dist']:.4f}m")
print(f"  평균: {np.mean(best_dists):.4f}m")

print(f"\n{'='*70}")
print(f"✅ [V12-Lite] 완료!")
print(f"{'='*70}")
print(f"주요 변경사항:")
print(f"  1. Type/Result만 Entity Embedding (희소성 문제 해결)")
print(f"  2. Player → is_same 피처 활용")
print(f"  3. Team → is_home 피처 활용")
print(f"  4. 작은 초기화 (std=0.01)")
print(f"  5. Embedding Dropout (안정화)")
print(f"  6. One-Hot 대비 차원 축소 (25→18 embedding)")
print(f"  7. 관계 학습: Pass↔Cross, Carry↔Duel 등")
print(f"{'='*70}")